In [2]:
import pickle
from collections import defaultdict
from collections import Counter
import json

def pprint(dct):
    print(json.dumps(
    dct,
    sort_keys=True,
    indent=4,
    separators=(',', ': ')
    ))

# Load matched reads
with open('comp1_to_count.pickle', 'rb') as handle, open('comp2_to_count.pickle', 'rb') as handle2:
    comp1 = pickle.load(handle)
    comp2 = pickle.load(handle2)
    

class ConnectedComponents:

    isolated_source = list()
    isolated_target = list()

    def __init__(self, min_count=1):
        self.__source = list()
        self.__target = list()
        self.final_components = dict()
        self.number_of_components = int()
        self.THRESHOLD = min_count

    def add_edge(self, source_node, target_node, pair_count):
        if pair_count >= self.THRESHOLD:
            self.__source.append(source_node)
            self.__target.append(target_node)
        else:
            self.isolated_source.append(source_node)
            self.isolated_target.append(target_node)

    def construct(self):
        __leaders = defaultdict(lambda: None)
        __groups = defaultdict(set)

        def __find(x):
            l = __leaders[x]
            if l is not None:
                l = __find(l)
                __leaders[x] = l
                return l
            return x

        def __union(x, y):
            lx, ly = __find(x), __find(y)
            if lx != ly:
                __leaders[lx] = ly

        for i in range(len(self.__source)):
            __union(self.__source[i], self.__target[i])

        for x in __leaders:
            __groups[__find(x)].add(x)

        for component_id, (k, v) in enumerate(__groups.items(), start=1):
            self.final_components[component_id] = v

        # Adding isolated components
#         last_key = max(self.final_components.keys()) + 1
#         for i in range(len(self.isolated_source)):
#             self.final_components[last_key] = [self.isolated_source[i],self.isolated_target[i]]
#             last_key += 1


        self.number_of_components = len(self.final_components)

    def get_components_dict(self):
        return self.final_components

    def dump_to_tsv(self, file_name):
        with open(file_name, 'w') as tsvWriter:
            for compID, nodes in self.final_components.items():
                nodes = ','.join(map(str, nodes))
                tsvWriter.write(f"{compID}\t{nodes}\n")

    def __del__(self):
        del self
        
pairsCountFile = "singlePartioning_aggressive_cDBG75_pairsCount.tsv"
originalComponentsFile = "cDBG_SRR11015356_k75.unitigs.fa.components.csv"

CUTOFF = 3
print(f"CUTOFF={CUTOFF}")
edges = []
with open(pairsCountFile, 'r') as pairsCountReader:
    next(pairsCountReader)  # skip header
    for line in pairsCountReader:
        edges.append(tuple(map(int, line.strip().split())))

components = ConnectedComponents(min_count=CUTOFF)
for edge in edges:
    components.add_edge(*edge)

components.construct()
number_of_final_components = components.number_of_components

numberOfOriginalComponents = 0
with open(originalComponentsFile) as origCompReader:
    for line in origCompReader:
        numberOfOriginalComponents += 1

final_components = components.get_components_dict().copy()
gathered_originalComponents_list = list()
for k, v in final_components.items():
    for origComp in v:
        gathered_originalComponents_list.append(origComp)
        
gathered_originalComponents = set(gathered_originalComponents_list)

assert len(gathered_originalComponents_list) == len(gathered_originalComponents)

print(f"gathered_originalComponents={len(gathered_originalComponents)}")

originalCompsSet = set(range(1, numberOfOriginalComponents+1))

assert len(originalCompsSet) == numberOfOriginalComponents

isoloatedComponents_set = originalCompsSet - gathered_originalComponents

print(f"number of isolated components={len(isoloatedComponents_set)}")

assert len(isoloatedComponents_set) + len(gathered_originalComponents) == numberOfOriginalComponents
print(f"total Number Of Components: {len(isoloatedComponents_set) + len(gathered_originalComponents)}")

# Adding the isolated components to the final components
last_key = max(final_components.keys()) + 1
for i, isolatedComp in enumerate(isoloatedComponents_set, start=last_key):
    final_components[i] = [isolatedComp]

listOfOriginalComponentsInFinalComponents = list()
for k, v in final_components.items():
    for c in v:
        listOfOriginalComponentsInFinalComponents.append(c)

assert len(listOfOriginalComponentsInFinalComponents) == len(set(listOfOriginalComponentsInFinalComponents))
print("---------------")

print(f"Number of final components: {len(final_components)}")



all_ln = [len(v) for k,v in final_components.items()]
freqs_ln = sorted(dict(Counter(all_ln)).items())
print("Distribution {Number of original components: Occurrence count}")
print(freqs_ln)

print("\n------------------------\n")

finalComp_to_matchedReads = dict()
for finalCompID, originalComps in final_components.items():
    finalComp_to_matchedReads[finalCompID] = 0
    for origComp in originalComps:
        finalComp_to_matchedReads[finalCompID] += comp1.get(origComp, 0)
        finalComp_to_matchedReads[finalCompID] += comp2.get(origComp, 0)
        
print("FinalComp: Number of matched reads")
print(Counter(finalComp_to_matchedReads).most_common(10))
print(Counter(finalComp_to_matchedReads).most_common()[-10:])

noMatchedReads=0
for k,v in finalComp_to_matchedReads.items():
    if v == 0:
        noMatchedReads += 1

print(f"Number of empty final components {noMatchedReads}")

CUTOFF=3
gathered_originalComponents=224043
number of isolated components=1682288
total Number Of Components: 1906331
---------------
Number of final components: 1767937
Distribution {Number of original components: Occurrence count}
[(1, 1682288), (2, 73100), (3, 8538), (4, 2305), (5, 843), (6, 388), (7, 203), (8, 98), (9, 77), (10, 32), (11, 20), (12, 17), (13, 9), (14, 9), (15, 2), (16, 5), (17, 1), (20, 1), (32434, 1)]

------------------------

FinalComp: Number of matched reads
[(2, 112086830), (5517, 18627), (109329, 17979), (10894, 17251), (71415, 16376), (26175, 15318), (3677, 13859), (40014, 13747), (13838, 13551), (8957, 13056)]
[(1767916, 0), (1767918, 0), (1767923, 0), (1767926, 0), (1767927, 0), (1767932, 0), (1767933, 0), (1767934, 0), (1767935, 0), (1767937, 0)]
Number of empty final components 1184787
